base class for Gates 

In [190]:
import numpy as np
# Define the base class for Gates 
class Gate: 
    def forward(self): 
        raise NotImplementedError 
    def backward(self): 
        raise NotImplementedError  

AddGate

In [191]:
# Example of an AddGate class inheriting from the Gate class 
class AddGate(Gate): 
    def forward(self, x, y): 
        self.x = x 
        self.y = y 
        return x + y 
    def backward(self, dz): 
        dx = dz * np.ones_like(self.x) 
        dy = dz * np.ones_like(self.y) 
        return dx, dy 

 MultiplyGate

In [192]:
# Example of a MultiplyGate class inheriting from the Gate class
class MultiplyGate(Gate):
    def forward(self, x, y):
        self.x = x
        self.y = y
        return x * y

    def backward(self, dz):
        dx = dz * self.y
        dy = dz * self.x
        return dx, dy

Linear activation function

In [193]:

# Example of a Linear activation function
class LinearActivation(Gate):
    def forward(self, x):
        self.x = x
        return x

    def backward(self, dz):
        dx = dz
        return dx

ReLU activation function

In [194]:
# Example of a ReLU activation function
class ReLUActivation(Gate):
    def forward(self, x):
        self.x = x
        return np.maximum(0, x)

    def backward(self, dz):
        dx = dz * np.where(self.x > 0, 1, 0)
        return dx

Sigmoid activation function

In [195]:
# import numpy as np

# class SigmoidActivation:
#     def __init__(self):
#         self.x = None

#     def forward(self, x):
#         self.x = x
#         return 1 / (1 + np.exp(-x))

#     def backward(self, dz):
#         sigmoid_x = 1 / (1 + np.exp(-self.x))
#         dx = dz * sigmoid_x * (1 - sigmoid_x)
#         return dx
class SigmoidActivation:
    def __init__(self):
        self.sigmoid_x = None

    def forward(self, x):
        self.sigmoid_x = 1 / (1 + np.exp(-x))
        return self.sigmoid_x

    def backward(self, dz):
        if self.sigmoid_x is None:
            raise ValueError("Forward method must be called before backward method.")
        dx = dz * self.sigmoid_x * (1 - self.sigmoid_x)
        return dx


Softmax activation function

In [196]:
# Example of a Softmax activation function
class SoftmaxActivation(Gate):
    def forward(self, x):
        self.x = x
        exp_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
        return exp_x / np.sum(exp_x, axis=-1, keepdims=True)

    def backward(self, dz):
        softmax_x = self.forward(self.x)
        dx = dz * softmax_x * (1 - softmax_x)
        return dx

Tanh activation function

In [197]:
# Example of a Tanh activation function
class TanhActivation(Gate):
    def forward(self, x):
        self.x = x
        return np.tanh(x)

    def backward(self, dz):
        tanh_x = np.tanh(self.x)
        dx = dz * (1 - tanh_x ** 2)
        return dx

 Binary Cross-Entropy (BCE) loss function

In [198]:
# Example of Binary Cross-Entropy (BCE) loss function
class BinaryCrossEntropyLoss(Gate):
    def forward(self, y_pred, y_true):
        self.y_pred = y_pred
        self.y_true = y_true
        return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

    def backward(self):
        dx = (self.y_pred - self.y_true) / (self.y_pred * (1 - self.y_pred))
        return dx

L2 loss function

In [199]:
# Example of L2 loss function
class L2Loss(Gate):
    def forward(self, y_pred, y_true):
        self.y_pred = y_pred
        self.y_true = y_true
        return 0.5 * np.mean((y_pred - y_true) ** 2)

    def backward(self):
        dx = self.y_pred - self.y_true
        return dx

Implementing Computational Graph / Model 

In [200]:
class Model:
    def __init__(self, layers_dim, activation_func, loss):
        self.layers_dim = layers_dim
        self.activation_func = activation_func
        self.loss = loss
        self.parameters = {}
        self.activations = {}
        self.gradients = {}

        # Initialize weights and biases using Xavier initialization
        for i in range(1, len(layers_dim)):
            prev_dim = layers_dim[i - 1]
            curr_dim = layers_dim[i]
            self.parameters["W" + str(i)] = np.random.randn(curr_dim, prev_dim) * np.sqrt(1 / prev_dim)
            self.parameters["b" + str(i)] = np.zeros((curr_dim, 1))  # Initialize biases as column vectors

    def predict(self, X):
        self.activations["A0"] = X

        for i in range(1, len(self.layers_dim)):
            prev_a = self.activations["A" + str(i - 1)]
            W = self.parameters["W" + str(i)]
            b = self.parameters["b" + str(i)]

            activation_func = self.activation_func()  # Create an instance of the activation function
            Z = np.dot(W, prev_a) + b
            A = activation_func.forward(Z)

            self.activations["A" + str(i)] = A
            self.activations["Z" + str(i)] = Z

        return self.activations["A" + str(len(self.layers_dim) - 1)]

    def train(self, X, y, num_epochs, learning_rate):
        for epoch in range(num_epochs):
            # Forward propagation
            A = self.predict(X)

            # Compute loss
            loss = self.loss.forward(A, y)
            mse = np.mean((A - y) ** 2)

            # Backward propagation
            dA = self.loss.backward()
            self.gradients["dA" + str(len(self.layers_dim) - 1)] = dA

            for i in reversed(range(1, len(self.layers_dim))):
                activation_func = self.activation_func()  # Create an instance of the activation function
                A_prev = self.activations["A" + str(i - 1)]
                W = self.parameters["W" + str(i)]
                b = self.parameters["b" + str(i)]

                Z = self.activations["Z" + str(i)]  # Retrieve Z from stored activations
                activation_func.forward(Z)  # Call forward method to compute self.sigmoid_x
                dZ = activation_func.backward(self.gradients["dA" + str(i)])  # Pass Z to backward method

                dW = np.dot(dZ, A_prev.T)
                db = np.sum(dZ, axis=1, keepdims=True)

                self.gradients["dA" + str(i - 1)] = np.dot(W.T, dZ)
                self.gradients["dW" + str(i)] = dW
                self.gradients["db" + str(i)] = db

            # Update parameters
            for i in range(1, len(self.layers_dim)):
                self.parameters["W" + str(i)] -= learning_rate * self.gradients["dW" + str(i)]
                self.parameters["b" + str(i)] -= learning_rate * self.gradients["db" + str(i)]

            print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss}, MSE: {mse}")
            print("Weights:")
            for i in range(1, len(self.layers_dim)):
                print(f"Layer {i}:")
                print(f"W{i}:")
                print(self.parameters["W" + str(i)])
            print()


 Example usage
 

In [201]:
# Example usage:
layers_dim = [2, 3, 1]  # Example: 2 input units, 3 units in the hidden layer, 1 output unit
activation_func = SigmoidActivation  # Use Sigmoid activation function
loss = L2Loss()  # Use L2 loss function
model = Model(layers_dim, activation_func, loss)

# Assuming X and y are your input and output data
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([[0], [1], [1], [0]])

num_epochs = 100  # Specify the number of epochs
learning_rate = 0.1  # Specify the learning rate
model.train(X.T, y.T, num_epochs, learning_rate)

Epoch 1/100, Loss: 0.13048606660432394, MSE: 0.26097213320864787
Weights:
Layer 1:
W1:
[[ 0.4843833   0.71454329]
 [ 0.05071271 -0.37626411]
 [-0.18416556 -0.15371826]]
Layer 2:
W2:
[[ 0.56295672  0.18003536 -0.02740132]]

Epoch 2/100, Loss: 0.13001537012266154, MSE: 0.2600307402453231
Weights:
Layer 1:
W1:
[[ 0.48418205  0.71425049]
 [ 0.05050028 -0.37649457]
 [-0.18413291 -0.15368523]]
Layer 2:
W2:
[[ 0.55684282  0.17551582 -0.03192046]]

Epoch 3/100, Loss: 0.1295804311074689, MSE: 0.2591608622149378
Weights:
Layer 1:
W1:
[[ 0.48400456  0.71398133]
 [ 0.05030115 -0.37671159]
 [-0.1840963  -0.15364817]]
Layer 2:
W2:
[[ 0.55097296  0.17117422 -0.03626414]]

Epoch 4/100, Loss: 0.12917901450787012, MSE: 0.25835802901574023
Weights:
Layer 1:
W1:
[[ 0.48384977  0.71373478]
 [ 0.05011452 -0.37691594]
 [-0.18405627 -0.15360764]]
Layer 2:
W2:
[[ 0.5453408   0.16700588 -0.04043679]]

Epoch 5/100, Loss: 0.12880894768235565, MSE: 0.2576178953647113
Weights:
Layer 1:
W1:
[[ 0.48371665  0.71350983

# Gates w msh 3arf ash8lo

In [32]:
import numpy as np

# Define the base class for Gates
class Gate:
    def forward(self):
        raise NotImplementedError

    def backward(self):
        raise NotImplementedError

# Example of an AddGate class inheriting from the Gate class
class AddGate(Gate):
    def forward(self, x, y):
        self.x = x
        self.y = y
        return x + y

    def backward(self, dz):
        dx = dz * np.ones_like(self.x)
        dy = dz * np.ones_like(self.y)
        return dx, dy

# Example of a MultiplyGate class inheriting from the Gate class
class MultiplyGate(Gate):
    def forward(self, x, y):
        self.x = x
        self.y = y
        return x * y

    def backward(self, dz):
        dx = dz * self.y
        dy = dz * self.x
        return dx, dy

# Define activation functions
class ActivationFunctions:
    @staticmethod
    def linear(x):
        return x

    @staticmethod
    def relu(x):
        return np.maximum(0, x)

    @staticmethod
    def sigmoid(x):
        return 1 / (1 + np.exp(-x))

    @staticmethod
    def softmax(x):
        exp_values = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exp_values / np.sum(exp_values, axis=1, keepdims=True)

    @staticmethod
    def tanh(x):
        return np.tanh(x)

# Define loss functions
class LossFunctions:
    @staticmethod
    def binary_cross_entropy(y_true, y_pred):
        return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

    @staticmethod
    def l2_loss(y_true, y_pred):
        return np.mean(np.square(y_true - y_pred))

# Define the Model class
class Model:
    def __init__(self, layers_dim, activation_func, loss):
        self.layers_dim = layers_dim
        self.activation_func = activation_func
        self.loss = loss

    def predict(self, X):
        # Forward pass
        z1 = MultiplyGate().forward(X, np.array([[1], [1]]))  # AND gate
        a1 = ActivationFunctions.relu(z1)
        z2 = MultiplyGate().forward(a1, np.array([[2]]))  # Output layer
        y_pred = ActivationFunctions.sigmoid(z2)
        return y_pred

    def train(self, X, y, num_epochs, learning_rate):
        for epoch in range(num_epochs):
            # Forward pass
            z1 = MultiplyGate().forward(X, np.array([[1], [1]]))  # AND gate
            a1 = ActivationFunctions.relu(z1)
            z2 = MultiplyGate().forward(a1, np.array([[2]]))  # Output layer
            y_pred = ActivationFunctions.sigmoid(z2)

            # Compute loss
            loss = self.loss(y, y_pred)

            # Backward pass
            dz2 = y_pred - y
            dz2 *= y_pred * (1 - y_pred)
            da1, dw2 = MultiplyGate().backward(dz2)
            dz1 = da1 * (a1 > 0)
            dx, dw1 = MultiplyGate().backward(dz1)

            # Update weights
            learning_rate = 0.01
            dw1 *= learning_rate
            dw2 *= learning_rate

            # Print loss
            if epoch % 100 == 0:
                print(f"Epoch {epoch}, Loss: {loss}")


